In [60]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf

from sklearn.preprocessing import OrdinalEncoder


In [61]:
X=pd.read_csv("train.csv")
X_Test=pd.read_csv("test.csv")

In [62]:
X

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [63]:
X.dropna(axis=0, subset=["Survived"], inplace=True)
y=X.Survived
X.drop(['Survived'], axis=1, inplace=True)

In [64]:
from sklearn.model_selection import train_test_split

X_Train, X_Valid, y_Train, y_Valid = train_test_split(X,y,
                                                      train_size=0.8, test_size=0.2,
                                                      random_state= 1)

In [65]:
X_Train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
301,302,3,"McCoy, Mr. Bernard",male,NaN,2,0,367226,23.2500,NaN,Q
309,310,1,"Francatelli, Miss. Laura Mabel",female,30.0,0,0,PC 17485,56.9292,E36,C
516,517,2,"Lemore, Mrs. (Amelia Milley)",female,34.0,0,0,C.A. 34260,10.5000,F33,S
120,121,2,"Hickman, Mr. Stanley George",male,21.0,2,0,S.O.C. 14879,73.5000,NaN,S
570,571,2,"Harris, Mr. George",male,62.0,0,0,S.W./PP 752,10.5000,NaN,S


In [66]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [67]:
#Categorical columns in the training data
categorical_Columns = [col for col in X_Train.columns if X_Train[col].dtype == "object"]

#Categorical columns that can be conveted to ordinal columns
good_Columns = [col for col in categorical_Columns if
                set(X_Valid[col]).issubset(set(X_Train[col]))]

#Problematic columns that will be droped from the dataset
bad_Columns = list(set(categorical_Columns)-set(good_Columns))

print("Will be encoded: ", good_Columns)
print("\nWill be droped: ",bad_Columns)

Will be encoded:  ['Sex', 'Embarked']

Will be droped:  ['Ticket', 'Name', 'Cabin']


In [76]:
#Draop categorical columns that won't be encoded
good_X_Train = X_Train.drop(bad_Columns, axis=1)
good_X_Valid = X_Valid.drop(bad_Columns, axis=1)
good_X_Test = X_Test.drop(bad_Columns, axis=1)

#Apply ordinal encoder
ordinal_encoder = OrdinalEncoder()
good_X_Train[good_Columns]=ordinal_encoder.fit_transform(X_Train[good_Columns])
good_X_Valid[good_Columns]=ordinal_encoder.transform(X_Valid[good_Columns])
good_X_Test[good_Columns]=ordinal_encoder.transform(X_Test[good_Columns])

#Make the NaN values in the remaining columns equals the median
for val in good_X_Train.columns[good_X_Train.isnull().any(axis=0)]:
    good_X_Train[val].fillna(good_X_Train[val].mean(), inplace=True)

for val in good_X_Valid.columns[good_X_Valid.isnull().any(axis=0)]:
    good_X_Valid[val].fillna(good_X_Valid[val].mean(), inplace=True)

for val in good_X_Test.columns[good_X_Test.isnull().any(axis=0)]:
    good_X_Test[val].fillna(good_X_Test[val].mean(), inplace=True)

In [73]:
#model
model = RandomForestRegressor(n_estimators=400, random_state=1)
model.fit(good_X_Train, y_Train)
preds= model.predict(good_X_Valid)

In [74]:
print("MAE:")
print(mean_absolute_error(y_Valid, preds))

MAE:
0.27265363128491626


In [77]:
df_Test = model.predict_proba(good_X_Test)[:,1]
df_Test[df_Test >.49] = 1
df_Test[df_Test <= .49] = 0
df_Test = pd.Series(df_Test, index=good_X_Test.index, name="Predictions")

In [78]:
df_Test

array([0.0275, 0.4025, 0.2575, 0.0275, 0.58  , 0.0775, 0.695 , 0.1475,
       0.8325, 0.0725, 0.005 , 0.4725, 0.91  , 0.255 , 0.905 , 0.865 ,
       0.0175, 0.4775, 0.4275, 0.35  , 0.5075, 0.7825, 0.9125, 0.5225,
       0.9725, 0.0175, 0.9875, 0.455 , 0.545 , 0.285 , 0.0525, 0.12  ,
       0.4525, 0.15  , 0.4625, 0.4525, 0.3925, 0.4325, 0.0175, 0.245 ,
       0.255 , 0.54  , 0.0375, 0.88  , 0.9   , 0.0625, 0.7625, 0.0075,
       0.945 , 0.4625, 0.4025, 0.235 , 0.885 , 0.95  , 0.31  , 0.0575,
       0.005 , 0.0125, 0.0225, 0.94  , 0.0225, 0.315 , 0.02  , 0.7225,
       0.74  , 0.8675, 0.7425, 0.43  , 0.75  , 0.9475, 0.6625, 0.0225,
       0.4075, 0.7025, 0.94  , 0.43  , 0.0075, 0.6875, 0.0325, 0.6625,
       0.8875, 0.35  , 0.4775, 0.005 , 0.0875, 0.22  , 0.81  , 0.605 ,
       0.7975, 0.9625, 0.51  , 0.0025, 0.945 , 0.0075, 0.67  , 0.015 ,
       0.9075, 0.14  , 0.51  , 0.025 , 0.9275, 0.04  , 0.0075, 0.01  ,
       0.63  , 0.0275, 0.155 , 0.0075, 0.0625, 0.025 , 0.2025, 0.79  ,
      